In [2]:
!pip install schedule

In [35]:
import urllib.request
import requests
import urllib.parse
import xml.etree.ElementTree as ET
import pandas as pd
import os
import html
import schedule #install 필수
import time
from bs4 import BeautifulSoup

In [33]:
#ID,Secret은  https://developers.naver.com/apps/#/list에서 API를 등록 후 ID,Secret를 아래 Id,secret에 넣으시면 됩니다.
Id= 'Your_ID'
secret= 'Your_secret'


CSV_FILE_PATH = 'naver_news_data.csv'  #csv파일 이름


#이전에 API 사용 없이는 수백개 데이터를 가져 올 수 있었지만 현재는 회 당 최대 100 일당 최대 20000으로 제한이 걸렸습니다.
#그래서 스캐줄 또는 코드를 통해서 시간마다 자동으로 저장을 하여 데이터를 가져오거나 일일이 작동 시켜야합니다.


def url_make(keyword, display=10, start=1, sort='sim'):
    '''
    Keyword: 키워드

    display 출력개수 min:1 max:100

    start 출력지점 min:1 max:1000

    sort 정렬기준 sim(정확도) date(최신순)


    '''


    # 각 정보를 담을 빈 리스트 초기화
    titles = []
    describes = []
    datetimes = []
    links = []
    encText = urllib.parse.quote(keyword)
    # URL 생성 (display 파라미터 위치 수정)
    url = f"https://openapi.naver.com/v1/search/news.xml?query={encText}&display={display}&start={start}&sort={sort}"
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", Id)
    request.add_header("X-Naver-Client-Secret", secret)

    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if rescode == 200:
        response_body = response.read()
        xml_string = response_body.decode('utf-8')
        root = ET.fromstring(xml_string)

        for item in root.findall('./channel/item'): # 루프 한 번만 실행
            # 각 요소 찾기
            title_element = item.find('title')
            link_element = item.find('link')
            describe_element = item.find('description')
            date_element = item.find('pubDate')

            # Title 처리
            if title_element is not None and title_element.text is not None:
                raw_title = title_element.text
                # HTML 태그 제거 및 HTML 엔티티 변환 (&quot; 등)
                clean_title = html.unescape(raw_title.replace('<b>', '').replace('</b>', '')).replace('&quot;', '"')
                titles.append(clean_title)
            else:
                titles.append(None)

            # Link 처리
            if link_element is not None and link_element.text is not None:
                links.append(link_element.text)
            else:
                links.append(None)

            # Description 처리
            if describe_element is not None and describe_element.text is not None:
                raw_describe = describe_element.text
                clean_describe = html.unescape(raw_describe.replace('<b>', '').replace('</b>', ''))
                describes.append(clean_describe)
            else:
                describes.append(None)

            # pubDate 처리
            if date_element is not None and date_element.text is not None:
                datetimes.append(date_element.text) # 날짜 형식 변환은 필요시 추가
            else:
                datetimes.append(None)

    else:
        print(f"Error Code: {rescode}")
        # API 오류 시 빈 DataFrame 반환 또는 예외 발생 등 처리 가능
        return pd.DataFrame() # 빈 DataFrame 반환 예시

    news_df = pd.DataFrame({
        'Title': titles,
        'Describe': describes,
        'Link': links,
        'DateTime': datetimes
    })

    if not os.path.exists(CSV_FILE_PATH):
        news_df.to_csv(CSV_FILE_PATH, index=False, mode='w', encoding='utf-8-sig')
    else:
        news_df.to_csv(CSV_FILE_PATH, index=False, mode='a', header=False, encoding='utf-8-sig')


    # 생성된 DataFrame 반환
    return news_df

In [ ]:
def job():
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] 작업을 실행합니다: 키워드='대선'")
    try:
        url_make('대선', display=100,sort='date') # 원하는 값 설정 가능 (범위는 위에서 제공)
    except Exception as e:
        print(f"작업 실행 중 오류 발생: {e}")

# 매 30분마다 job 함수 실행 예약
schedule.every(30).minutes.do(job)

#최소 1회 실행
job()

# 예약된 작업들을 계속 확인하고 실행하는 루프
start_time = time.time()
run_seconds = 48 * 60 * 60 # 48시간

while time.time() - start_time < run_seconds:
    schedule.run_pending()
    time.sleep(1)

schedule.clear()
print("수집 종료.")

In [37]:
def get_naver_news_body(url):
    """
    네이버 뉴스 URL을 받아 본문 텍스트를 추출하는 함수

    Args:
        url (str): 네이버 뉴스 기사 URL (mnews.naver.com 형태)

    Returns:
        str: 추출된 기사 본문 텍스트. 실패 시 None 반환.
    """
    if not url or not url.startswith("https://n.news.naver.com/mnews/article/"):
        print("올바른 네이버 뉴스 기사 URL 형식이 아닙니다.")
        return None

    headers = {
        # 웹사이트가 봇으로 인식하여 차단하는 것을 피하기 위해 일반적인 브라우저 User-Agent 설정
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }


    # URL에 HTTP GET 요청 보내기
    response = requests.get(url, headers=headers, timeout=10)
    # HTTP 상태 코드가 200 (성공) 인지 확인
    response.raise_for_status() # 200이 아니면 HTTPError 발생

    # 응답받은 HTML 내용을 BeautifulSoup으로 파싱
    soup = BeautifulSoup(response.text, 'html.parser')

    #본문 위치 찾기
    article_body_tag = soup.find('article', id='dic_area')

    if article_body_tag:
        # 불필요한 태그 제거 (예: 기자 정보, 광고 등) - 선택 사항
        # 예시: 기사 본문 내 '기자 프로필' 부분 제거 (실제 구조에 따라 수정 필요)
        unwanted_elements = article_body_tag.find_all('div', attrs={'class':'journalistcard_module'})
        for element in unwanted_elements:
            element.decompose() # 태그 제거

        # 이미지 설명이나 캡션 등 특정 부분 제외가 필요하다면 추가 로직 구현 -선택사항
        unwanted_spans = article_body_tag.find_all('span', attrs={'class':'end_photo_org'})
        for span in unwanted_spans:
            span.decompose()

        # 태그 내부의 텍스트만 추출
        # separator='\n' : 각 텍스트 조각 사이에 줄바꿈 추가
        # strip=True : 각 텍스트 조각의 앞뒤 공백 제거
        body_text = article_body_tag.get_text(separator='\n', strip=True)
        return body_text

In [ ]:
news_data=pd.read_csv('naver_news_data.csv')
link=news_data['Link']
contents = []
for i in link:
    content=get_naver_news_body(i)
    contents.append(content)

news_data['Contents'] = contents